In [1]:
import torch
from torch import nn

# Exercice 1

$$
    y = 1 \cdot X - 1 + \epsilon
    \newline
    x \in \mathbb{R}^{60 \times 1}
    \newline
    \epsilon \sim N(\mu = 0, \sigma = 0.1)
$$

In [20]:
# Exercice 1
X = torch.normal(0, 1, (60, 1))
w = torch.tensor([1.0])
eps = torch.normal(0, 0.1, (1, 60))
y = X @ w - 1 + eps

features = X
labels = y.reshape(-1, 1)

In [ ]:
net = nn

# Exercice 2

In [2]:
import torchvision
from torchvision import transforms

In [ ]:
data_train = torchvision.datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
data_test = torchvision.datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)